## Deep CBOW 实现

In [2]:
from collections import defaultdict
import time
import random
import dynet as dy
import numpy as np

# Functions to read in the corpus
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]
def read_dataset(filename):
  with open(filename, "r") as f:
    for line in f:
      tag, words = line.lower().strip().split(" ||| ")
      yield ([w2i[x] for x in words.split(" ")], t2i[tag])

# Read in the data
train = list(read_dataset("data/classes/train.txt"))
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("data/classes/test.txt"))
nwords = len(w2i)
ntags = len(t2i)

# Start DyNet and define trainer
model = dy.Model()
trainer = dy.AdamTrainer(model)

In [9]:
# Define the model
EMB_SIZE = 64
HID_SIZE = 64
HID_LAY = 2
W_emb = model.add_lookup_parameters((nwords, EMB_SIZE)) # Word embeddings
W_h = [model.add_parameters((HID_SIZE, EMB_SIZE if lay == 0 else HID_SIZE)) for lay in range(HID_LAY)]
b_h = [model.add_parameters((HID_SIZE)) for lay in range(HID_LAY)]
W_sm = model.add_parameters((ntags, HID_SIZE))          # Softmax weights
b_sm = model.add_parameters((ntags))                      # Softmax bias

In [10]:
def calc_scores(words):
  dy.renew_cg()
  h = dy.esum([dy.lookup(W_emb, x) for x in words])
  for W_h_i, b_h_i in zip(W_h, b_h):
    h = dy.tanh( dy.parameter(W_h_i) * h + dy.parameter(b_h_i) )
  return dy.parameter(W_sm) * h + dy.parameter(b_sm)

for ITER in range(100):
  # Perform training
  random.shuffle(train)
  train_loss = 0.0
  start = time.time()
  for words, tag in train:
    my_loss = dy.pickneglogsoftmax(calc_scores(words), tag)
    train_loss += my_loss.value()
    my_loss.backward()
    trainer.update()
  print("iter %r: train loss/sent=%.4f, time=%.2fs" % (ITER, train_loss/len(train), time.time()-start))
  # Perform training
  test_correct = 0.0
  for words, tag in dev:
    scores = calc_scores(words).npvalue()
    predict = np.argmax(scores)
    if predict == tag:
      test_correct += 1
  print("iter %r: test acc=%.4f" % (ITER, test_correct/len(dev)))

iter 0: train loss/sent=1.5452, time=0.58s
iter 0: test acc=0.3801
iter 1: train loss/sent=1.2578, time=0.58s
iter 1: test acc=0.3810
iter 2: train loss/sent=1.0238, time=0.58s
iter 2: test acc=0.3760
iter 3: train loss/sent=0.7901, time=0.58s
iter 3: test acc=0.3982
iter 4: train loss/sent=0.5872, time=0.58s
iter 4: test acc=0.3873
iter 5: train loss/sent=0.4111, time=0.59s
iter 5: test acc=0.3869
iter 6: train loss/sent=0.2899, time=0.61s
iter 6: test acc=0.3968
iter 7: train loss/sent=0.2010, time=0.62s
iter 7: test acc=0.3891
iter 8: train loss/sent=0.1444, time=0.64s
iter 8: test acc=0.3891
iter 9: train loss/sent=0.1000, time=0.63s
iter 9: test acc=0.3896
iter 10: train loss/sent=0.0744, time=0.67s
iter 10: test acc=0.3733
iter 11: train loss/sent=0.0484, time=0.63s
iter 11: test acc=0.3661
iter 12: train loss/sent=0.0374, time=0.64s
iter 12: test acc=0.3860
iter 13: train loss/sent=0.0353, time=0.62s
iter 13: test acc=0.3968
iter 14: train loss/sent=0.0299, time=0.57s
iter 14: t